In [33]:
import pandas as pd
from  bs4 import BeautifulSoup
import requests
from warnings import filterwarnings
filterwarnings('ignore')
import datetime as dt
from time import sleep

### Краснодар

In [37]:
def parse_minfinkubani_ru():
    domain = 'https://minfinkubani.ru'
    url = 'https://minfinkubani.ru/deposit_funds/selection_parameters.php'
    
    s = requests.Session()
    r = s.get(url, verify=False)
    
    soup = BeautifulSoup(r.text)
    parags = soup.find_all('div', {'class': 'item_mews_title'})
    parsed = []

    for p in parags:
        ext_url = domain + p.a.attrs['href']
        ext_content = BeautifulSoup(requests.get(ext_url, verify=False).text)
        d = ext_url.split('=')[2]
        if dt.datetime.strptime(d, '%d.%m.%Y') >= dt.datetime.today():
            parsed.append([d, ext_url])
            
    s.close()
    parsed = pd.DataFrame(parsed, columns=['date', 'url'])
    
    return parsed

In [38]:
parsed_minfinkubani = parse_minfinkubani_ru()
parsed_minfinkubani

,date,url


### Банк России

In [39]:
def parse_cbr_ru():
    url = 'https://cbr.ru/DKP/DepoParams/'
    df = pd.read_html(url)[0].iloc[:, 0]
    tod = dt.date.today().strftime("%d.%m.%Y")
    df = df[pd.to_datetime(df, format='%d.%m.%Y') >= tod]
    df = pd.DataFrame({'date':df.to_list(), 'url':[url] * len(df)})
    return df

In [40]:
parsed_cbr = parse_cbr_ru()
parsed_cbr

,date,url


### Moex

In [41]:
def month2num(month):
    month_dict = {
        'января': 1,
        'февраля': 2,
        'марта': 3,
        'апреля': 4,
        'мая':5,
        'июня': 6,
        'июля': 7,
        'августа': 8,
        'сентября': 9,
        'октября': 10,
        'ноября': 11,
        'декабря': 12
    }
    return month_dict[month]

In [42]:
def moex_parser(kwords):
    domain = 'https://www.moex.com/'
    url = 'https://www.moex.com/ru/news/?ncat=114'
    
    r = requests.get(url).text
    soup = BeautifulSoup(r)
    parags = soup.find_all('a', {'class': 'news-list__link'})
    
    parsed = []

    for p in parags:
        full_url = domain + p.attrs['href']
        text = p.text.lower()

        for w in kwords:
            if w not in text:
                break
        else:
            action_date = p.text.lower().split('состоится')[0]
            action_date =\
                f'{action_date.split()[0]}.{month2num(action_date.split()[1])}.{action_date.split()[2]}'
            action_date = dt.datetime.strptime(action_date, '%d.%m.%Y')
            if action_date >= dt.datetime.today():
                parsed.append([dt.datetime.strftime(action_date, '%d.%m.%Y'),
                               full_url
                              ])

    parsed = pd.DataFrame({'date':[x[0] for x in parsed],
                           'url':[x[1] for x in parsed]})
    
    return parsed

In [43]:
words_fedkazna = ['федеральн', 'казначей']
parsed_fedkazna = moex_parser(words_fedkazna)
parsed_fedkazna

,date,url


In [44]:
words_pfr = ['пенсион', 'фонд']
parsed_pfr = moex_parser(words_pfr)
parsed_pfr

,date,url


### Aggregation

In [45]:
df = parsed_cbr.append(parsed_minfinkubani)

In [46]:
df = df.append(parsed_fedkazna)
df = df.append(parsed_pfr)

In [47]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df.sort_values('date')

,date,url


### Writing

In [48]:
df.to_excel('parsed_deposit_auctions.xlsx')